In [37]:
import os
import json
import tensorflow as tf

In [38]:
def create_dir(directory):
    if not os.path.exists(directory):
        os.makedirs(directory)

def save_tensor_as_bin(tensor, filepath):
    tensor = tensor.numpy() if hasattr(tensor, 'numpy') else tensor
    tensor.tofile(filepath)

def save_json(data, filepath):
    with open(filepath, 'w') as f:
        json.dump(data, f, indent=4)

In [39]:
def layer_wise_dump_tf(model, input_tensor, save_dir):
    # Ensure save directory structure
    create_dir(save_dir)
    create_dir(f"{save_dir}/inputs")
    create_dir(f"{save_dir}/outputs")
    create_dir(f"{save_dir}/weights")
    create_dir(f"{save_dir}/configurations")

    model_config = {
        "model_name": model.name,
        "layer_count": len(model.layers),
        "input_shape": list(model.input_shape) if hasattr(model, 'input_shape') else "Unknown",
        "output_shape": list(model.output_shape) if hasattr(model, 'output_shape') else "Unknown",
        "layer_details": []
    }

    x = input_tensor
    for layer in model.layers:
        layer_name = layer.name

        # Save input tensor
        input_filepath = f"{save_dir}/inputs/{layer_name}_input.bin"
        save_tensor_as_bin(x, input_filepath)

        # Save weights
        weight_filepaths = []
        if hasattr(layer, 'weights') and layer.weights:
            for weight in layer.weights:
                weight_name = weight.name.split("/")[-1].replace(":", "_")
                weight_filepath = f"{save_dir}/weights/{layer_name}_{weight_name}.bin"
                save_tensor_as_bin(weight.numpy(), weight_filepath)
                weight_filepaths.append(weight_filepath)

        # Compute layer output
        x = layer(x)

        # Save output tensor
        output_filepath = f"{save_dir}/outputs/{layer_name}_output.bin"
        save_tensor_as_bin(x, output_filepath)

        # Gather layer attributes
        layer_info = {
            "name": layer_name,
            "type": layer.__class__.__name__,
            "input_shape": list(layer.input_shape) if hasattr(layer, 'input_shape') else "Unknown",
            "output_shape": list(layer.output_shape) if hasattr(layer, 'output_shape') else "Unknown",
            "input_bin": input_filepath,
            "output_bin": output_filepath,
            "weight_files": weight_filepaths,
        }
        if hasattr(layer, 'kernel_size'):
            layer_info["kernel_size"] = layer.kernel_size
        if hasattr(layer, 'strides'):
            layer_info["strides"] = layer.strides
        if hasattr(layer, 'padding'):
            layer_info["padding"] = layer.padding
        if hasattr(layer, 'activation') and layer.activation:
            layer_info["activation"] = layer.activation.__name__

        # Add layer details to the configuration
        model_config["layer_details"].append(layer_info)

    # Save the model configuration to a JSON file
    save_json(model_config, f"{save_dir}model_configuration.json")

    return model_config

In [40]:
model_path = 'cifar10_resnet_model.h5'
model = tf.keras.models.load_model(model_path)
input_tensor = tf.random.normal([1, 32, 32, 3]) 
save_dir = '../Data'

In [41]:
layer_wise_dump_tf(model, input_tensor, save_dir)


{'model_name': 'sequential_1',
 'layer_count': 8,
 'input_shape': [None, 32, 32, 3],
 'output_shape': [None, 10],
 'layer_details': [{'name': 'conv2d_3',
   'type': 'Conv2D',
   'input_shape': 'Unknown',
   'output_shape': 'Unknown',
   'input_bin': '../Data/inputs/conv2d_3_input.bin',
   'output_bin': '../Data/outputs/conv2d_3_output.bin',
   'weight_files': ['../Data/weights/conv2d_3_kernel.bin',
    '../Data/weights/conv2d_3_bias.bin'],
   'kernel_size': (3, 3),
   'strides': (1, 1),
   'padding': 'same',
   'activation': 'relu'},
  {'name': 'max_pooling2d_2',
   'type': 'MaxPooling2D',
   'input_shape': 'Unknown',
   'output_shape': 'Unknown',
   'input_bin': '../Data/inputs/max_pooling2d_2_input.bin',
   'output_bin': '../Data/outputs/max_pooling2d_2_output.bin',
   'weight_files': [],
   'strides': (2, 2),
   'padding': 'valid'},
  {'name': 'conv2d_4',
   'type': 'Conv2D',
   'input_shape': 'Unknown',
   'output_shape': 'Unknown',
   'input_bin': '../Data/inputs/conv2d_4_input.bi